# Using Semantic Kernel with MCP Servers

In this notebook, we'll connect a simple SK Agent to an MCP Server.

## 📋 Prerequisites & Configuration

### 🔧 **Environment Setup**
1. **Copy the environment file**:
   ```bash
   cp .env.example .env
   ```

2. **Edit `.env` file** with your actual credentials:
   - Azure OpenAI endpoint, API key, and deployment name
   - OR regular OpenAI API key

3. **Start the MCP server** (see instructions below)

### 🗄️ **Database Setup**
The MCP server uses a SQLite database (`contoso.db`) that should be in the same directory.

### ⚡ **Quick Start**
The notebook will automatically load configuration from your `.env` file.

First, let's start our MCP server.

Open a new terminal and run:

using uv:
```shell
cd 03-semantic-kernel-mcp\
uv run --prerelease=allow mcp_server.py
```

using pip:
```shell
pip install fastmcp
cd .\03-semantic-kernel-mcp\
python .\mcp_server.py
```

The server should come up like this:

```
INFO:     Started server process [49488]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
```

Great, ready to go, let's connect SK to it:

In [ ]:
import os
from dotenv import load_dotenv
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin

# Load environment variables from .env file
load_dotenv()

print("🔧 Setting up MCP plugin...")

# Set up the SSE plugin for the MCP service.
contoso_mcp_plugin = MCPSsePlugin(
    name="ContosoMCP",
    description="Contoso MCP Plugin",
    url=os.getenv("MCP_SERVER_URL", "http://localhost:8000/sse"),
    headers={"Content-Type": "application/json"},
    timeout=30,
)

# Connect to MCP server
await contoso_mcp_plugin.connect()
print("✅ MCP plugin connected successfully!")

print("\n🤖 Setting up Semantic Kernel agent...")

# Configure Azure OpenAI service
service = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
)

# Create the agent with MCP plugin
agent = ChatCompletionAgent(
    service=service,
    name="ChatBot",
    instructions="You are a helpful assistant. You can use multiple tools to find information "
    "and answer questions. Review the tools available under the ContosoMCP plugin "
    "and use them as needed. You can also ask clarifying questions if the user is not clear.",
    plugins=[contoso_mcp_plugin],
)

print("✅ Setup complete! Agent ready to help with customer inquiries.")

# Same as prior in our workshop 
thread: ChatHistoryAgentThread = None
user_messages = [
    "I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?",
    "My customer id is 42",
    ]

for user_message in user_messages:
    print("*** User:", user_message)
    response = await agent.get_response(messages=user_message, thread=thread)
    thread = response.thread
    print("*** Agent:", response.content)



ServiceInitializationError: chat_deployment_name is required.

In [ ]:
# Test the agent with customer inquiry
print("🗣️  Starting customer conversation...\n")

# Same as prior in our workshop 
thread: ChatHistoryAgentThread = None
user_messages = [
    "I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?",
    "My customer id is 42",
]

for user_message in user_messages:
    print(f"👤 User: {user_message}")
    
    response = await agent.get_response(messages=user_message, thread=thread)
    thread = response.thread
    print(f"🤖 Agent: {response.content}\n")
    print("-" * 80 + "\n")

print("✅ Conversation complete!")